In [ ]:
#Callback
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import BatchNormalization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Input, Dropout

data = pd.read_csv('/content/ABNB_stock_data.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst = True)
data.set_index('Date', inplace=True)
data.head()

df1 = pd.DataFrame(data['Close'])
data = df1.values
train_test_split = int(np.ceil(0.65 * len(data)))
train_data = data[:train_test_split]
test_data = data[train_test_split:]
train_data.shape, test_data.shape

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])


x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(LSTM(units=50, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=5, activation='relu'))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1)

# Training
model.fit(x_train, y_train, batch_size=16, epochs=100, callbacks=[reduce_lr, early_stopping])

y_train = scaler.inverse_transform(y_train)
y_train_predict = model.predict(x_train)
y_train_predict = scaler.inverse_transform(y_train_predict)

lstm_r2 = r2_score(y_train, y_train_predict)
lstm_mse = mean_squared_error(y_train, y_train_predict)
lstm_mae = mean_absolute_error(y_train, y_train_predict)

print('MSE:', lstm_mse) 
print('MAE:', lstm_mae) 
print('R2 Score:', lstm_r2) 


In [ ]:
#FINETUNING
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from scipy.stats import randint, uniform

# Load and preprocess data
data = pd.read_csv('ABNB_stock_data.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data.set_index('Date', inplace=True)

df1 = pd.DataFrame(data['Close'])
data = df1.values

scaler = StandardScaler()
data = scaler.fit_transform(data)

train_test_split = int(np.ceil(0.65 * len(data)))
train_data = data[:train_test_split]
test_data = data[train_test_split:]

x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Define the LSTM model function
def create_lstm_model(units=50, dropout_rate=0.2, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=5))
    model.add(Dense(units=1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Wrap the model with KerasRegressor
model = KerasRegressor(build_fn=create_lstm_model, verbose=0)

# Define the parameter space for RandomizedSearchCV
param_dist = {
    'model__units': [32, 50, 64, 100],
    'model__dropout_rate': uniform(0.1, 0.5),  # Dropout values between 0.1 and 0.5
    'batch_size': [16, 32, 64],
    'epochs': [50, 100, 200],
    'optimizer': ['adam', 'rmsprop']
}

# Use RandomizedSearchCV to tune hyperparameters
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                   n_iter=10, cv=3, verbose=2, random_state=42)

random_search.fit(x_train, y_train)

# Get the best parameters and train the best model
print("Best parameters found: ", random_search.best_params_)

# Predict using the best model
best_model = random_search.best_estimator_

y_train_predict = best_model.predict(x_train)
y_train_predict = scaler.inverse_transform(y_train_predict.reshape(-1, 1))
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))

# Evaluate the best model
lstm_r2 = r2_score(y_train, y_train_predict)
lstm_mse = mean_squared_error(y_train, y_train_predict)
lstm_mae = mean_absolute_error(y_train, y_train_predict)

print('MSE:', lstm_mse)
print('MAE:', lstm_mae)
print('R2 Score:', lstm_r2)